In [104]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score

In [105]:
df = pd.read_csv('log2.csv') 
df

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,allow,177,94,83,2,30,1,1
1,56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9
2,6881,50321,43265,50321,allow,238,118,120,2,1199,1,1
3,50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,allow,25358,6778,18580,31,16,13,18
...,...,...,...,...,...,...,...,...,...,...,...,...
65527,63691,80,13237,80,allow,314,192,122,6,15,4,2
65528,50964,80,13485,80,allow,4680740,67312,4613428,4675,77,985,3690
65529,54871,445,0,0,drop,70,70,0,1,0,1,0
65530,54870,445,0,0,drop,70,70,0,1,0,1,0


In [106]:
unique_values = df['Action'].unique()
print(unique_values)

['allow' 'drop' 'deny' 'reset-both']


In [107]:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

le = LabelEncoder()
df['Action'] = le.fit_transform(df['Action'])
df

mapping = dict(zip(le.classes_, le.transform(le.classes_)))

print('Mapping:\n', mapping)


Mapping:
 {'allow': 0, 'deny': 1, 'drop': 2, 'reset-both': 3}


In [108]:
X = df.drop(columns=['Action']) 
y = df['Action'] 

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size = 0.2,
                                                    random_state = 42)


In [109]:
svm_model = SVC()
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)

In [110]:
svm_accuracy = accuracy_score(y_test, svm_predictions)
print('SVM Accuracy:', svm_accuracy)

SVM Accuracy: 0.5756466010528725


In [111]:
print('SVM Classification Report:')
print(classification_report(y_test, svm_predictions))

SVM Classification Report:
              precision    recall  f1-score   support

           0       0.58      1.00      0.73      7545
           1       0.00      0.00      0.00      2994
           2       0.00      0.00      0.00      2562
           3       0.00      0.00      0.00         6

    accuracy                           0.58     13107
   macro avg       0.14      0.25      0.18     13107
weighted avg       0.33      0.58      0.42     13107



C:\Users\austi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\austi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\austi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [112]:
sgd_model = SGDClassifier()
sgd_model.fit(X_train, y_train)
sgd_predictions = sgd_model.predict(X_test)

In [113]:
sgd_accuracy = accuracy_score(y_test, sgd_predictions)
print('SGD Accuracy:', sgd_accuracy)

SGD Accuracy: 0.7959868772411688


In [114]:
print('SGD Classification Report:')
print(classification_report(y_test, sgd_predictions))

SGD Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      7545
           1       0.53      0.99      0.69      2994
           2       0.00      0.00      0.00      2562
           3       0.00      0.00      0.00         6

    accuracy                           0.80     13107
   macro avg       0.38      0.50      0.42     13107
weighted avg       0.69      0.80      0.73     13107



C:\Users\austi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\austi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\austi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [115]:
y_test_binarized = label_binarize(y_test, classes = [0, 1, 2, 3])

svm_probs = svm_model.decision_function(X_test)
sgd_probs = sgd_model.decision_function(X_test)

n_classes = 4
fpr_svm = dict()
tpr_svm = dict()
roc_auc_svm = dict()

fpr_sgd = dict()
tpr_sgd = dict()
roc_auc_sgd = dict()

plt.figure(figsize=(12, 8))

colors = ['orange', 'navy', 'green', 'red']

for i in range(n_classes):
    plt.subplot(2, 2, i + 1)  
    fpr_svm[i], tpr_svm[i], _ = roc_curve(y_test_binarized[:, i], svm_probs[:, i])
    roc_auc_svm[i] = auc(fpr_svm[i], tpr_svm[i])
    
    fpr_sgd[i], tpr_sgd[i], _ = roc_curve(y_test_binarized[:, i], sgd_probs[:, i])
    roc_auc_sgd[i] = auc(fpr_sgd[i], tpr_sgd[i])
    
    plt.plot(fpr_svm[i], 
             tpr_svm[i],
             color = colors[i],
             lw = 2,
             label = f'SVM (AUC = {roc_auc_svm[i]:.2f})')
    
    plt.plot(fpr_sgd[i],
             tpr_sgd[i],
             color = colors[i],
             linestyle = '--', 
             lw = 2,
             label = f'SGD (AUC = {roc_auc_sgd[i]:.2f})')
    
    plt.plot([0, 1], 
             [0, 1],
             color = 'gray', 
             lw = 2,
             linestyle = '--')
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for Class {i}')
    plt.legend(loc = 'lower right')

plt.tight_layout()
plt.show()
